In [ ]:
# Copyright 2022-2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Import "**Edge Application**" to "**Console**"

This notebook explains the workflow for importing "**Edge Application**" to "**Console**". <br>
<br>
Instructions are described in [README.md](./README.md).

## Imports

In [ ]:
import base64
import errno
import json
import os
from pathlib import Path

import jsonschema
from console_access_library.client import Client

## Load Configurations

Load the configuration file and set the variables.

In [ ]:
def validate_symlink(path: Path):
    if path.is_symlink():
        raise OSError(
            errno.ELOOP,
            "Symbolic link is not supported. Please use real folder or file",
            f"{path}",
        )


configuration_path = Path("./configuration.json")
validate_symlink(configuration_path)

# Load configuration file
with open(configuration_path, "r") as f:
    json_load = json.load(f)

configuration_schema_path = Path("./configuration_schema.json")
validate_symlink(configuration_schema_path)

# Load configuration schema file
with open(configuration_schema_path, "r") as f:
    json_schema = json.load(f)

# Validate configuration
jsonschema.validate(json_load, json_schema)

# Set compiled_flg(0:wasm, 1:aot):
compiled_flag = "0"

# Set app entry point:
entry_point = "main"

## Convert "**Edge Application**" to string for Base64

In [ ]:
# file encode to base64
def convert_file_to_b64_string(file_path):
    with open(file_path, "rb") as f:
        return base64.b64encode(f.read())


ppl_path = Path(json_load["ppl_file"].replace(os.path.sep, "/"))
validate_symlink(ppl_path)

file_content = convert_file_to_b64_string(ppl_path)

file_name = os.path.basename(ppl_path)

print(json_load["ppl_file"] + " is loaded.")

## Load an instance of "**Console Access Library**" Client

Run this cell to restore instance variable 'client_obj' created on a notebook of Set up "**Console Access Library**".

In [ ]:
%store -r
client_obj: Client = client_obj

## Import "**Edge Application**"

In [ ]:
# Get an instance of Deployment API
deployment_obj = client_obj.get_deployment()

param_dict = json_load.copy()

param_dict["compiled_flg"] = compiled_flag
param_dict["entry_point"] = entry_point
param_dict["file_name"] = file_name
param_dict["file_content"] = file_content
del param_dict["ppl_file"]

# Call an API to import Vision and Sensing Application into Console for AITRIOS
try:
    response = deployment_obj.import_device_app(**param_dict)
except Exception as e:
    # EXCEPTION
    raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
print(
    "Start to import the Vision and Sensing Application."
    + " \n\tapp_name: "
    + json_load["app_name"]
    + "\n\tversion_number: "
    + json_load["version_number"]
)

## Get "**Edge Application**" status

Converting takes time.<br>

To complete the conversion, ensure that the conversion status is **`Conversion completed`**.

After you start the conversion, run the following code cell to check the status :

In [ ]:
def get_device_app_status(app_name: str, version_number: str):
    # Status of Vision and Sensing Application on Console
    status_dictionary = {
        "0": "Importing completed (Before conversion)",
        "1": "Converting...",
        "2": "Conversion completed",
        "3": "Conversion failed",
    }
    # Flag for import check
    import_flag = False
    # Call an API to get Vision and Sensing Application info
    try:
        response = deployment_obj.get_device_apps()
    except Exception as e:
        # EXCEPTION
        raise e

    # response error check
    if "result" in response and response["result"] != "SUCCESS":
        # ERROR
        raise ValueError("ERROR", response)

    # SUCCESS
    # Create output list
    apps = response.get("apps", [])
    for app in apps:
        if "name" in app and app["name"] == app_name:
            versions = app.get("versions", [])
            for version in versions:
                if "version" in version and version["version"] == version_number:
                    import_flag = True
                    version_status = version.get("status", "")
                    break
            if import_flag:
                break
    if import_flag:
        return status_dictionary.get(
            version_status, "Unknown status '" + version_status + "'"
        )
    else:
        raise Exception(
            "Vision and Sensing Application is not found. "
            + " \n\tapp_name: "
            + app_name
            + "\n\tversion_number: "
            + version_number
        )


get_status = get_device_app_status(
    app_name=json_load["app_name"], version_number=json_load["version_number"]
)
print(
    get_status
    + " \n\tapp_name: "
    + json_load["app_name"]
    + "\n\tversion_number: "
    + json_load["version_number"]
)